In [1]:
import os
import sys

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns 

# Reading the DATA: 

In [2]:
# The data comes from 
os.listdir("../../project_data/")

['GSE131512_metaData.xlsx',
 'GSE131512_normalTPM.txt',
 'validation_bc_meta.xlsx',
 'pnas_patient_info.csv',
 'validation_normal_meta.xlsx',
 'GSE131512_cancerTPM.txt']

## Metadata

In [3]:
# Metadata from the GEO page
filepath = "../../project_data/GSE131512_metaData.xlsx"
metadata = pd.read_excel(filepath)

# metadata for validation datasets from the github (not sure which samples it is referencing)
filepath = "../../project_data/validation_bc_meta.xlsx"
validation_bc_meta = pd.read_excel(filepath)
filepath = "../../project_data/validation_normal_meta.xlsx"
validation_normal_meta = pd.read_excel(filepath)

# additional metadata about cancer subtypes for the 96 cancer patients
filepath = "../../project_data/pnas_patient_info.csv"
patient_info = pd.read_csv(filepath)

In [85]:
# checking the shapes of the metadata to see if they fit expectation
metadata.shape,patient_info.shape, validation_normal_meta.shape, validation_bc_meta.shape

((128, 9), (96, 32), (78, 3), (83, 4))

In [54]:
# Adding a chemo duration, and time from chemo end to recurrence fields
dt_end = pd.to_datetime(patient_info['datechemoend'], format="%d/%m/%Y")
dt_start = pd.to_datetime(patient_info['datechemostart'], format="%d/%m/%Y")
dt_recurrence = pd.to_datetime(patient_info['daterecurrence'], format="%d/%m/%Y")
patient_info['chemo_duration'] = (dt_end - dt_start).dt.days
patient_info['recurrence_time'] = (dt_recurrence - dt_end).dt.days

In [58]:
# Joining the two metadatas together)
metadata = pd.concat([metadata, 
                      patient_info[['cancertype', 'cancerstage_cat', 'chemo_duration', 'recurrence_time',  'chemo', 'recurStatus']]], 
                     axis=1, join='outer')
# setting the recurrence time for those that don't have recurrence to nan
metadata.loc[metadata.recurStatus == 'N', 'recurrence_time'] = np.nan

# writing the result out in a csv
metadata.to_csv('../data/metadata.csv')

## TPM data

In [22]:
normal_df = pd.read_csv("../../project_data/GSE131512_normalTPM.txt", delimiter = '\t')
cancer_df = pd.read_csv("../../project_data/GSE131512_cancerTPM.txt", delimiter = '\t')

In [23]:
normal_df.shape, cancer_df.shape

((60675, 32), (60675, 96))